# Samuel Watkins, 3032132676

# HW 6: Homebrew Computer Vision
## Due Monday Apr 2, 2018 at 2 PM

1. Download the [zip file](https://www.dropbox.com/s/cst9awcjpp08k33/50_categories.tar.gz). Look at some of the images, noting that there are 50 classes in 4244 images (e.g. "goldfish", “llama”, “speed-boat”, ...). Caution: it’s a pretty large file (~208M).
2. Write a set of methods that takes as input one of these images, and then computes real-numbered features as the return. You should produce at least 15 features.
3. Based on the feature set for each image, build a random forest classifier. Produce metrics on your estimated error rates using cross-validation. How much better is this than the expectation with random guessing? What are the 3 most important features?
4. Make sure your final classifier can run on a directory of different images, where a call like `run_final_classifier("/new/directory/path/")` on a directory that contains files like `validation1.jpg`, `validation2.jpg`, etc. will produce an output file that looks like:  
```
filename              predicted_class  
``` 
` `-----------------------------------------------------------------
```
validation1.jpg       unicorn  
validation2.jpg       camel  
```

    We will have a validation set to test how good your classifier is.

# Function to Extract Features from an Image
We have written a function that takes in a path to an image and extracts 33 different features to be used to train a random forest classifier.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import corner_harris,peak_local_max,canny,corner_peaks,blob_doh
from skimage.segmentation import slic
from skimage.color.colorconv import rgb2grey,grey2rgb
from skimage.filters import frangi,gaussian,threshold_li
from skimage.transform import rescale,hough_line,hough_line_peaks
from skimage.measure import shannon_entropy

In [2]:
def extractImageFeatures(pathToImage,smallImageSize=16000.0):
    """
    Feature Extraction Function
    
        This function takes a path to an image, reads the image into a numpy array, 
        and does various analyses on the image to extract features. These analyses range from aspect ratio 
        to Shannon entropy. The output is a numpy array that stores each extracted feature as a single number. 
        For many of the features, the image will be scaled down by a factor determined by the parameter smallImageSize,
        which is done to speed up the code.
    
    Parameters
    ----------
    pathToImage : str
        The path to the image from which we will extract features from.
    smallImageSize : float
        The area in pixels of the scaled-down image from which certain features will be extracted from.
        By default, this is set to be 16000 (units of pixels**2)
    
    Returns
    -------
    features : numpy.ndarray
        An array of floats containing each feature that was extracted from the image.
    
    """
    # read in the image as a numpy array
    imageArray = plt.imread(pathToImage).astype("float")
    
    # if the image is already greyscale, then convert it to and RGB array
    if len(imageArray.shape)<3:
        imageArray = grey2rgb(imageArray)
    # convert the image to greyscale
    greyImgArr = rgb2grey(imageArray)
    # calculate the scale factor that will scale the image to the size specificed by smallImageSize
    scaleFactor = np.sqrt(smallImageSize/np.prod(greyImgArr.shape))
    # scale down the greyscale and RGB images 
    imgScaled = rescale(greyImgArr,scaleFactor,mode = "constant")
    imgScaledColor = rescale(imageArray,scaleFactor,mode = "constant")
    # apply a gaussian filter to the greyscale image for smoothing
    imgScaledGauss = gaussian(imgScaled)
    
    # calculate the area, height, width, and aspect ratio of the image
    imgSize = np.prod(greyImgArr.shape)
    imgHeight = greyImgArr.shape[0]
    imgWidth = greyImgArr.shape[1]
    aspectRatio = imgWidth/imgHeight
    
    # find the average value of all of the RGB channels
    avgAllChans = np.mean(imageArray)
    
    # find the average value of each of the RGB channels
    avgRedChan = np.mean(imageArray[:,:,0])
    avgGreenChan = np.mean(imageArray[:,:,1])
    avgBlueChan = np.mean(imageArray[:,:,2])
    
    # calculate the ratios of the average value of each channel
    ratioRedBlue = avgRedChan/avgBlueChan
    ratioBlueGreen = avgBlueChan/avgGreenChan
    ratioRedGreen = avgRedChan/avgGreenChan
    
    # apply a corner detection algorithm to count how many corners there are in the image
    corners = corner_harris(imgScaled)
    numCorners = len(corner_peaks(corners))
    
    # calculate the number of peaks in the image
    peaks = peak_local_max(imgScaled)
    numPeaks = len(peaks)
    
    # apply segmentation algorithm to find the number of segments in the image
    segments = slic(imgScaled)
    numSegments = np.max(segments)
    
    # apply an edge detection algorithm to calculate the length of the edges
    edges = canny(frangi(imgScaledGauss))
    edgeLength = np.sum(edges)
    if edgeLength == 0:
        edgeLength+=1
    
    # zoom in to the middle of the picture to calculate the ratio of edges in the middle to the total edge length
    edgesZoomed = edges[int(imgScaled.shape[0]/4):-int(imgScaled.shape[0]/4),
                        int(imgScaled.shape[1]/4):-int(imgScaled.shape[1]/4)]
    edgesInMiddle = np.sum(edgesZoomed)/edgeLength
    
    # apply a line detection algorithm to count the approximate number of lines in the edges
    hspace,angles,distances = hough_line(edges)
    if np.sum(hspace)>0:
        numLines = len(hough_line_peaks(hspace,angles,distances)[0])
    else:
        numLines = 0
    
    # take the ratio of number of lines to edge length
    ratioLinesEdges = numLines/edgeLength
    
    # calculate how many times a vertical and horizontal line through the image crosses a detected edge
    horizontalEdges = edges[int(edges.shape[0]/2)]
    verticalEdges = edges[:,int(edges.shape[1]/2)]
    horizontalEdgeCrossings = sum(horizontalEdges[:-1]!=horizontalEdges[1:])
    verticalEdgeCrossings = sum(verticalEdges[:-1]!=verticalEdges[1:])

    # take ratios of corners, peaks, segments, and edge length
    ratioCornersPeak = numCorners/numPeaks
    ratioCornersSegments = numCorners/numSegments
    ratioCornersEdges = numCorners/edgeLength
    ratioPeaksSegments = numPeaks/numSegments
    ratioPeaksEdges = numPeaks/edgeLength
    ratioSegmentsEdges = numSegments/edgeLength
    
    # calculate the shannon entropy of the image
    shanent = shannon_entropy(imageArray)
    
    # apply a threshold algorithm to determine the foreground of the image and the size of the foreground
    thresh = threshold_li(imgScaled)
    foreground = imgScaled <=thresh
    foregroundSize = np.sum(foreground)
    
    # take the ratio of foreground size to edge length
    ratioForegroundEdge = foregroundSize/edgeLength
    
    # take the ratio of the edge length of the foreground to the edge length of the entire image
    edgeRatio = np.sum(canny(gaussian(foreground)))/edgeLength
    
    # find the average colors in the foreground
    avgForegroundRed = np.mean(imgScaledColor[:,:,0][foreground])
    avgForegroundGreen = np.mean(imgScaledColor[:,:,1][foreground])
    avgForegroundBlue = np.mean(imgScaledColor[:,:,2][foreground])
    
    # take the ratios of the average foreground colors
    ratioFGRedGreen = avgForegroundRed/avgForegroundGreen
    ratioFGRedBlue = avgForegroundRed/avgForegroundBlue
    ratioFGGreenBlue = avgForegroundGreen/avgForegroundBlue
    
    # store all values in an array
    features=np.array([aspectRatio,ratioRedBlue,ratioBlueGreen,
                        ratioRedGreen,numPeaks,numSegments,edgeLength,
                        horizontalEdgeCrossings,verticalEdgeCrossings,
                        ratioCornersPeak,ratioCornersSegments,ratioPeaksSegments,
                        ratioPeaksEdges,ratioSegmentsEdges,shanent,thresh,foregroundSize,
                        avgForegroundRed,avgForegroundGreen,avgForegroundBlue,
                        ratioFGRedGreen,ratioFGRedBlue,ratioFGGreenBlue,numLines,
                        ratioCornersEdges,avgAllChans,avgRedChan,avgGreenChan,avgBlueChan,
                        edgeRatio,edgesInMiddle,ratioLinesEdges,ratioForegroundEdge])
    
    # if any feature is nan or inf, set it to zero
    features[np.isnan(features)]=0.0
    features[np.isinf(features)]=0.0
    
    return features

In [ ]:
from skimage.feature import blob_doh,shape_index
from copy import deepcopy
from skimage.filters import try_all_threshold,threshold_minimum
from skimage.color import label2rgb
from skimage.segmentation import random_walker


pathToImage = "/home/sam/Documents/watkins-ay250-s2018-hw/hw_6/50_categories/bear/bear_0012.jpg"
imageArray = plt.imread(pathToImage).astype("float")
imageArray = grey2rgb(imageArray)
greyImgArr = rgb2grey(imageArray)
scaleFactor = np.sqrt(16000.0/np.prod(greyImgArr.shape))
imgScaledColor = rescale(imageArray,scaleFactor,mode = "constant")
imgScaled = rescale(greyImgArr,scaleFactor,mode = "constant")
threshval = threshold_otsu(gaussian(imgScaled))
imgBackground = gaussian(imgScaled) > threshval
imgForeground = deepcopy(imgScaled)
imgForeground[imgBackground] = 0
binary = gaussian(imgScaled) <=threshval
imgFilt = binary
edgePic = canny(frangi(gaussian(imgScaled)))

val = threshold_minimum(imgScaled)
binary = imgScaled <= val
plt.imshow(canny(gaussian(binary)))

plt.imshow(imgScaled[int(imgScaled.shape[0]/4):-int(imgScaled.shape[0]/4),int(imgScaled.shape[1]/4):-int(imgScaled.shape[1]/4)])

# blobs = blob_doh(gaussian(imgForeground),min_sigma=1.0,max_sigma=30,log_scale=False)

# nonEdgeBlobs = np.logical_and.reduce((np.all(blobs!=0,axis=1),blobs[:,0]!=imgScaled.shape[0]-1,blobs[:,1]!=imgScaled.shape[1]-1))

# blobs = blobs[nonEdgeBlobs]
# print(imgScaled.shape)
# print(len(blobs))
# print(blobs)

# fig, ax = plt.subplots(figsize=(9, 3))
# ax.imshow(imgScaled, interpolation='nearest')
# for blob in blobs:
#     y, x, r = blob
#     c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
#     ax.add_patch(c)

# Extract Features from All Images
Using the `extractImageFeatures` function, we will extract the features from all images in the dataset, and then split the dataset into training and test sets. We have used a 50/50 split of training and test sets.

In [3]:
from glob import glob
from sklearn.model_selection import train_test_split
from multiprocessing import Pool
from time import time

In [4]:
# initialize the test and training sets

pathToImageFolders = "50_categories/"
eachFolder = glob(pathToImageFolders+"*/")

train_size = 0.5 # ratio of training dataset to total dataset

X = list()
Y = list()

# open up 16 processes to extract features in parallel
num_processes = 16
pool = Pool(processes=num_processes)

starttime = time() # let's time how long this takes
# go through each folder and extract features from each image in the folder
for iFolder,folder in enumerate(eachFolder):
    print(f"Looking in folder {iFolder+1} of {len(eachFolder)} folders...")
    filesInFolder = glob(folder+"*.jpg")
    # extract image features from each file in the folder
    parallelFeatures = pool.map(extractImageFeatures,filesInFolder)
    # store the image features in X and the image categories in Y
    X.append(np.vstack(parallelFeatures))
    Y.append(np.repeat(folder[len(pathToImageFolders):-1],len(filesInFolder))) # image categories come from folder names

print(time()-starttime) # print the time elapsed

pool.terminate()
del pool

# put all of the features and categories into single arrays
X = np.vstack(X)
Y = np.concatenate(Y)

# split into test and train sets using the specified train_size
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,train_size=train_size,stratify=Y)


Looking in folder 1 of 50 folders...
Looking in folder 2 of 50 folders...
Looking in folder 3 of 50 folders...
Looking in folder 4 of 50 folders...
Looking in folder 5 of 50 folders...
Looking in folder 6 of 50 folders...
Looking in folder 7 of 50 folders...
Looking in folder 8 of 50 folders...
Looking in folder 9 of 50 folders...
Looking in folder 10 of 50 folders...
Looking in folder 11 of 50 folders...
Looking in folder 12 of 50 folders...
Looking in folder 13 of 50 folders...
Looking in folder 14 of 50 folders...
Looking in folder 15 of 50 folders...
Looking in folder 16 of 50 folders...
Looking in folder 17 of 50 folders...
Looking in folder 18 of 50 folders...
Looking in folder 19 of 50 folders...
Looking in folder 20 of 50 folders...
Looking in folder 21 of 50 folders...
Looking in folder 22 of 50 folders...
Looking in folder 23 of 50 folders...


/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars


Looking in folder 24 of 50 folders...
Looking in folder 25 of 50 folders...
Looking in folder 26 of 50 folders...


/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars


Looking in folder 27 of 50 folders...
Looking in folder 28 of 50 folders...
Looking in folder 29 of 50 folders...
Looking in folder 30 of 50 folders...
Looking in folder 31 of 50 folders...


...

/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars


Looking in folder 39 of 50 folders...
Looking in folder 40 of 50 folders...
Looking in folder 41 of 50 folders...
Looking in folder 42 of 50 folders...
Looking in folder 43 of 50 folders...
Looking in folder 44 of 50 folders...
Looking in folder 45 of 50 folders...


/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in long_scalars
/home/sam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in long_scalars


Looking in folder 46 of 50 folders...
Looking in folder 47 of 50 folders...
Looking in folder 48 of 50 folders...
Looking in folder 49 of 50 folders...
Looking in folder 50 of 50 folders...
355.85859656333923


/home/sam/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Build A Random Forest Classifier
Using the features extracted from the images in the training set, we will train our random forest classifier and see how accurate it is when applied to our test set.

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [6]:
# apply a random forest classifier to our training set
randforclf = RandomForestClassifier(n_estimators=50)
randforclf.fit(X_train,Y_train)
pred_rf = randforclf.predict(X_test)

In [15]:
# use the test set to calculate some accuracy metrics
print(f"Score: {metrics.accuracy_score(Y_test,pred_rf)}")
scores = cross_val_score(randforclf,X,Y,cv=5,groups=Y)
print(f"Accuracy from cross-validation: {np.mean(scores)} (+/- {np.std(scores)})")

Score: 0.30207351555136663
Accuracy from cross-validation: 0.3162290398474747 (+/- 0.0061073868962440315)


In [8]:
# what are the most important features?
randforclf.feature_importances_

array([0.08974201, 0.02913157, 0.03079588, 0.02727185, 0.02962395,
       0.02954755, 0.02575988, 0.02315035, 0.0224068 , 0.02814779,
       0.02700264, 0.02505088, 0.02451073, 0.02903308, 0.0506323 ,
       0.02624541, 0.02666196, 0.03035106, 0.02781869, 0.02736479,
       0.0266962 , 0.02764258, 0.02673588, 0.03106112, 0.0255416 ,
       0.02451733, 0.02741318, 0.02538112, 0.02816875, 0.03258217,
       0.02896516, 0.03626271, 0.02878302])

We see from the above cell that the most important features are the 1st, 15th, and 32nd features. These correspond to:

| Feature | Importance |  
| :------- | :---------- |  
| Aspect Ratio | 0.08974201 |  
| Shannon Entropy | 0.0506323 |  
| Ratio of Number of Lines to Edge Length | 0.03626271|  

# Compare to Random Guessing
Here, we apply a dummy classifier to the dataset that randomly guesses based on the distribution of categories in the training set. This is used to compare the performance of the random forest classifier that was trained using the extracted features.

In [9]:
from sklearn.dummy import DummyClassifier

In [10]:
# use a dummy classifier that randomly guesses what the image is to compare to our classifier
dummyclf = DummyClassifier(strategy="prior",random_state=42)
dummyclf.fit(X_train,Y_train)
dummypred_rf = dummyclf.predict(X_test)

In [11]:
# use the test set to calculate some accuracy metrics
print(f"Score: {metrics.accuracy_score(Y_test,dummypred_rf)}")
dummyscores = cross_val_score(dummyclf,X,Y,cv=5,groups=Y)
print(f"Accuracy from cross-validation: {np.mean(dummyscores)} (+/- {np.std(dummyscores)})")

Score: 0.12535344015080113
Accuracy from cross-validation: 0.12562240614744166 (+/- 0.0018389617183439597)


We see that, using cross validation, our **random forest classifier** had an accuracy of **31.6 $\pm$ 0.6 %**.  
A **dummy classifier** that randomly guesses the category based on the distribution of each category has an accuracy of **12.6 $\pm$ 0.2 %**. 

Thus, I was able to achieve an accuracy that was a factor of nearly **3 times better than randomly guessing**.

# Save the Random Forest Classifier
**NOTE**: I realize that Pickle is not a recommended way of storing models. However, since this is a short term assignment, I believe it should be fine to use Pickle on this model for the grader to use later. I was going to use Joblib, but I read [here](http://scikit-learn.org/stable/modules/model_persistence.html) that 

>Since a model internal representation may be different on two different architectures, dumping a model on one architecture and loading it on another architecture is not supported.  

So, I'm worried that using Joblib wouldn't allow the grader to load the file, if we end up having different architectures. Thus, I've elected to just use Pickle!

In [13]:
import pickle
with open('randforclf.pkl', 'wb') as f:
    pickle.dump(randforclf, f)

# Load the Random Forest Classifier and Run Predictions on a Directory
**NOTE**: You need to run the first two cells of this notebook to use the `run_final_classifier`, as we need to import the `extractImageFeatures` function and the various functions/modules it uses.

In [6]:
import pickle
from os.path import basename
from glob import glob
from multiprocessing import Pool

def run_final_classifier(pathToDirectory):
    """
    Function to Extract Features from All Images in a Directory
        
        This function takes in a path to a directory of images, extracts features from each images in the directory,
        prints the filenames and predicted category to a file called output.txt, and returns the predictions
        of the model.
    
    Parameters
    ----------
    pathToDirectory : str
        The path to the directory that contains all images from which we will extract features from.
    
    Returns
    -------
    pred_rf : numpy.ndarray
        An array that stores all of the predictions for the test data in the directory specified.
    
    """
    
    
    # open up 16 processes to extract features in parallel
    num_processes = 16
    pool = Pool(processes=num_processes)
    
    filesInFolder = glob(pathToDirectory+"*.jpg")
    filenames = [basename(x) for x in filesInFolder]
    
    # extract image features from each file in the folder
    X_test = pool.map(extractImageFeatures,filesInFolder)
    
    pool.terminate()
    
    X_test = np.vstack(X_test)
    
    with open('randforclf.pkl', 'rb') as f:
        randforclf = pickle.load(f)
    
    pred_rf = randforclf.predict(X_test)
    
    with open("output.txt","w") as f:
        f.write("filename\tpredicted_class\n")
        f.write("-----------------------------------------------------------------\n")
        for iFile,filename in enumerate(filenames):
            f.write(f"{filename}\t{pred_rf[iFile]}\n")
            
    return pred_rf

In [7]:
# change this path to the path to the validation dataset
pathToDirectory = "50_categories/airplanes/"

# run the final classifier on the validation dataset, which will
pred_rf = run_final_classifier(pathToDirectory) 